In [1]:
from pathlib import Path

from omegaconf import OmegaConf

In [23]:
logs_dir = Path('/checkpoint/linamezghani/offline-gcrl/logs/')

exp_name = "20220915-074847_walker_ours_s345_r0"

tmp_path = logs_dir / "multirun" / exp_name
cfg_path = tmp_path / list(tmp_path.iterdir())[0] /".hydra" / "config.yaml"

cfg = OmegaConf.load(cfg_path)

In [24]:
list(cfg_path.parent.iterdir())

[PosixPath('/checkpoint/linamezghani/offline-gcrl/logs/multirun/20220613-074149_walker_unsup-final-vec_edge-transTrue-subgoal-transTrue-with-velo_rb1M_s123_r0/2/.hydra/config.yaml'),
 PosixPath('/checkpoint/linamezghani/offline-gcrl/logs/multirun/20220613-074149_walker_unsup-final-vec_edge-transTrue-subgoal-transTrue-with-velo_rb1M_s123_r0/2/.hydra/hydra.yaml'),
 PosixPath('/checkpoint/linamezghani/offline-gcrl/logs/multirun/20220613-074149_walker_unsup-final-vec_edge-transTrue-subgoal-transTrue-with-velo_rb1M_s123_r0/2/.hydra/overrides.yaml')]

In [25]:
over_path = tmp_path / list(tmp_path.iterdir())[0] /".hydra" / "overrides.yaml"


In [26]:
print(over_path.read_text())

- +exp=walker_rnet_thresh
- main.suffix=unsup-final-vec_edge-trans$\{main.edge_transitions\}-subgoal-trans$\{main.subgoal_transitions\}-with-velo_rb1M
- main.load_from_dir=/checkpoint/linamezghani/offline-gcrl/trained_rnet/walker_memonly_thresh2_neg0_skip0.05_new-graph
- rnet.model.hidden_size=128
- rnet.model.comp_n_layers=4
- rnet.model.feat_size=128
- env.action_repeat=2
- main.reward=graph_sig
- replay_buffer.capacity=1000000
- replay_buffer.num_procs=20
- main.edge_transitions=True
- sac.policy.head.remove_velocity=False
- plot.type=wandb
- main.seed=123
- train.goal_strat=rb
- main.subgoal_transitions=True
- main.reward_sigm_temp=1
- eval.interval_epochs=20
- sac.optim.batch_size=2048
- replay_buffer.reward_scaling=0.5
- sac.optim.lr=0.0003
- sac.optim.entropy.alpha=0.01
- sac.optim.gamma=0.95
- optim.num_epochs=5001



In [27]:
ls /checkpoint/linamezghani/offline-gcrl/trained_rnet/walker_memonly_thresh2_neg0_skip0.05_new-graph

embs.pth  memory.npy  model.pth


In [ ]:
"walker_memonly_thresh2_neg0_skip0.05_new-graph"

In [33]:
import yaml
from deepdiff import DeepDiff

def yaml_as_dict(my_file):
    my_dict = {}
    with open(my_file, 'r') as fp:
        docs = yaml.safe_load_all(fp)
        for doc in docs:
            for key, value in doc.items():
                my_dict[key] = value
    return my_dict

if __name__ == '__main__':
    yaml_file1 = "/checkpoint/linamezghani/offline-gcrl/logs/multirun/20220613-074149_walker_unsup-final-vec_edge-transTrue-subgoal-transTrue-with-velo_rb1M_s123_r0/2/.hydra/config.yaml"
    yaml_file2 = cfg_path
    a = yaml_as_dict(yaml_file1)
    b = yaml_as_dict(yaml_file2)
    ddiff = DeepDiff(a, b, ignore_order=True)
    print(ddiff)

{'type_changes': {"root['main']['load_from_dir']": {'old_type': <class 'str'>, 'new_type': <class 'NoneType'>, 'old_value': '/checkpoint/linamezghani/offline-gcrl/trained_rnet/walker_memonly_thresh2_neg0_skip0.05_new-graph', 'new_value': None}, "root['main']['reward_sigm_temp']": {'old_type': <class 'int'>, 'new_type': <class 'float'>, 'old_value': 1, 'new_value': 1.0}}, 'dictionary_item_added': [root['exploration_buffer']['algo'], root['replay_buffer']['algo'], root['replay_buffer']['neg_action'], root['replay_buffer']['neg_goal'], root['replay_buffer']['cut_traj']], 'dictionary_item_removed': [root['rnet']['memory']['skip_traj']], 'values_changed': {"root['main']['suffix']": {'new_value': 'ours', 'old_value': 'unsup-final-vec_edge-trans${main.edge_transitions}-subgoal-trans${main.subgoal_transitions}-with-velo_rb1M'}, "root['main']['seed']": {'new_value': 345, 'old_value': 123}, "root['exploration_buffer']['data_dir']": {'new_value': '/checkpoint/${env:USER}/offline-gcrl/data/${env.i

In [34]:
for key in ddiff:
    print(key)
    for k in ddiff[key]:
        print(k)

type_changes
root['main']['load_from_dir']
root['main']['reward_sigm_temp']
dictionary_item_added
root['exploration_buffer']['algo']
root['replay_buffer']['algo']
root['replay_buffer']['neg_action']
root['replay_buffer']['neg_goal']
root['replay_buffer']['cut_traj']
dictionary_item_removed
root['rnet']['memory']['skip_traj']
values_changed
root['main']['suffix']
root['main']['seed']
root['exploration_buffer']['data_dir']
root['optim']['num_epochs']
root['rnet']['dataset']['thresh']
root['rnet']['train']['lr']
root['rnet']['train']['num_epochs']
root['rnet']['memory']['capacity']
root['rnet']['memory']['node_skip']
root['rnet']['memory']['edge_skip']


In [35]:
print(ddiff["values_changed"]["root['exploration_buffer']['data_dir']"])

{'new_value': '/checkpoint/${env:USER}/offline-gcrl/data/${env.id}/${exploration_buffer.algo}/buffer', 'old_value': '/checkpoint/${env:USER}/offline-gcrl/data/walker/proto/buffer'}


In [45]:
a['optim']

{'num_epochs': 5001}

In [44]:
b['optim']

{'num_epochs': 1001}

In [31]:
b['rnet']

{'model': {'feat_size': 128,
  'hidden_size': 128,
  'comparator': 'net',
  'n_layers': 3,
  'comp_n_layers': 4,
  'remove_velocity': True,
  'dims_to_keep': '${env.obs.no_velocity_size}'},
 'dataset': {'thresh': 2,
  'neg_thresh': 0,
  'in_traj_ratio': 0.5,
  'neg_ratio': 0.5,
  'symmetric': False,
  'valid_ratio': 0.05,
  'num_pairs': {'train': 500000, 'val': 100000}},
 'train': {'lr': 0.0003,
  'num_epochs': 100,
  'batch_size': 512,
  'num_workers': 32,
  'weight_decay': 1e-05},
 'memory': {'thresh': 0,
  'capacity': 10000,
  'node_skip': 0.05,
  'edge_skip': 0.05,
  'NN_batch_size': 1001,
  'directed': True}}

In [21]:
b['env']

{'id': 'walker',
 'success_thresh': 0.7,
 'max_episode_steps': 1000,
 'action_repeat': 2,
 'random_start_pos': False,
 'obs': {'type': 'vec',
  'state_size': 18,
  'vec_size': 24,
  'rgb_size': 64,
  'no_velocity_size': 15},
 'action_dim': 6,
 'frame_stack': 1,
 'obs_type': 'vec'}

In [22]:
a['env']

{'id': 'walker',
 'success_thresh': 0.7,
 'max_episode_steps': 1000,
 'action_repeat': 2,
 'random_start_pos': False,
 'obs': {'type': 'vec', 'state_size': 18, 'vec_size': 24, 'rgb_size': 64},
 'action_dim': 6,
 'obs_type': 'vec'}